In [ ]:
## Pulire digg

In [1]:
import networkx as nx
import pandas as pd
from tqdm import tqdm
import numpy as np

In [2]:
path_graph = '/Users/Cinus/University/Tesi/Dataset/digg_friends.csv'
path_prop = '/Users/Cinus/University/Tesi/Dataset/digg_votes1.csv'

In [20]:
with open(path_graph, 'r') as f:
    df_graph = pd.read_csv(f)

In [21]:
with open(path_prop, 'r') as f:
    df_prop = pd.read_csv(f, names=['time', 'node', 'item'])

In [22]:
selected_nodes = set(nodes[nodes.cnt > 19].index)
original2new = {original: new_id for original, new_id in zip(
    selected_nodes, range(len(selected_nodes)))}

In [23]:
df_prop_filtered = df_prop[df_prop.node.apply(selected_nodes.__contains__)]

In [24]:
df_prop_filtered['new_id'] = df_prop_filtered.node.apply(original2new.get)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [25]:
df_graph['new_id_i'] = df_graph.node.apply(original2new.get)
df_graph['new_id_j'] = df_graph.friend.apply(original2new.get)

In [31]:
df_graph_filtered = df_graph[~ 
                             (np.isnan(df_graph['new_id_i'])
                                       |
                                       np.isnan(df_graph['new_id_j'])
                                      )]


In [32]:
df_graph_filtered = df_graph_filtered.astype({'new_id_i': 'int', 'new_id_j': 'int'})

In [49]:
G = nx.DiGraph()

for mutual, i, j in df_graph_filtered[['mutual', 'new_id_i', 'new_id_j']].values:
    G.add_edge(i, j)
    if mutual:
        G.add_edge(j, i)

In [51]:
G.number_of_nodes()

15580

In [38]:
len(df_prop_filtered.item.unique())

3553

In [ ]:
pd.read_csv(path_prop)

In [ ]:
#estrarre nodi da tenere

from collections import Counter
count = Counter(list(df_prop[1]))
nodes = {node for node, c in count.items() if c > 19}
len(nodes)

In [ ]:
#estrarre nodi da tenere
def reduce_db(path_prop, reps=20):
    
    with open(path_prop, 'r') as f:
        df_prop = pd.read_csv(f, header=None)
    prop_nodes = list(df_prop[1])
    prop_nodes.sort()
    #print(prop_nodes)
    
    remain = []

    for i in range(len(prop_nodes)):
        count = 1
        for j in range(1, reps+1):
            try:
                if prop_nodes[i+j] == prop_nodes[i]:
                    count += 1
            except:
                pass
                #print('List finished: ', i+j)
        if count >= 20:
            remain.append(int(prop_nodes[i]))
    print('Reduction: from ', len(set(prop_nodes)), len(set(remain)))
    return set(remain)

In [ ]:
remain_set = reduce_db('/Users/Cinus/University/Tesi/Dataset/digg_votes1.csv')

In [ ]:
def extract():
    file_in = '/Users/Cinus/University/Tesi/Dataset/digg_friends.csv'
    file_out = '/Users/Cinus/University/Tesi/womg_git/Code/Library2.0/Input/Graph/graph_digg_edge_list.txt'
    path_prop = '/Users/Cinus/University/Tesi/Dataset/digg_votes1.csv'

    remain_set = reduce_db(path_prop)
    G = nx.Graph()
    
    with open(file_in) as ffile_in:
        df = pd.read_csv(ffile_in)
    count = 0
    for index in (range(int(df.last_valid_index()))):
        if df.loc[index]['node'] in remain_set:
            #print(index)
            count += 1
            G.add_edges_from([(int(df.loc[index]['node']), int(df.loc[index]['friend']))])
            if df.loc[index]['mutual'] == 1:
                G.add_edges_from([(int(df.loc[index]['friend']), int(df.loc[index]['node']))])
    #print(count)
    return G

In [ ]:
GG = extract()

In [ ]:
g = nx.read_edgelist('/Users/Cinus/University/Tesi/womg_git/Code/'
                     'Library2.0/Input/Graph/graph_digg_edge_list.txt',
                    create_using=nx.DiGraph(), nodetype=int)

In [ ]:
nx.info(g)

In [ ]:
nx_obj = nx.convert_node_labels_to_integers(g, first_label=0)

In [ ]:
sub_g = g.subgraph(remain_set).copy()

In [ ]:
sub_g.is_directed()

In [ ]:
sub_g.number_of_nodes()

In [ ]:
nx.info(sub_g)

In [ ]:
nx.write_edgelist(sub_g, '/Users/Cinus/University/Tesi/womg_git/Code/'
                     'Library2.0/Input/Graph/digg_edgelist.txt')

In [ ]:
Grafo = nx.read_edgelist('/Users/Cinus/University/Tesi/womg_git/Code/'
                     'Library2.0/Input/Graph/digg_edgelist.txt',
                        create_using=nx.DiGraph(), nodetype=int)

In [ ]:
Grafo.number_of_nodes()